In [1]:
!pip uninstall -y transformers -q
!pip install git+https://github.com/huggingface/transformers
!pip install livekit-agents langchain accelerate pydub faster-whisper
!pip install langchain-community
!!pip install elevenlabs

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-swoyuipq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-swoyuipq
  Resolved https://github.com/huggingface/transformers to commit 40dc11cd3eb4126652aa41ef8272525affd4a636
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=10906480 sha256=f346b8e146f9ef987a89218bacfd25b4ad6caa6d2ac11ee31db1edddcb3be2dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6zeezny/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transfo

['Requirement already satisfied: elevenlabs in /usr/local/lib/python3.12/dist-packages (2.26.1)',
 'Requirement already satisfied: httpx>=0.21.2 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (0.28.1)',
 'Requirement already satisfied: pydantic>=1.9.2 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (2.12.3)',
 'Requirement already satisfied: pydantic-core>=2.18.2 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (2.41.4)',
 'Requirement already satisfied: requests>=2.20 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (2.32.5)',
 'Requirement already satisfied: typing_extensions>=4.0.0 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (4.15.0)',
 'Requirement already satisfied: websockets>=11.0 in /usr/local/lib/python3.12/dist-packages (from elevenlabs) (15.0.1)',
 'Requirement already satisfied: anyio in /usr/local/lib/python3.12/dist-packages (from httpx>=0.21.2->elevenlabs) (4.12.0)',
 'Requirement already satisfied: cert

In [2]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=gen)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

/tmp/ipython-input-1455773634.py:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen)


In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",  """
You are an automated assistant responsible for contacting hotels and verifying their contact information.
Your goal is to politely speak with hotel staff, confirm or update the hotel's phone number, email address, and physical address, and record any corrected information using your available tools.
Always maintain a professional, friendly tone. Guide the conversation clearly and ask one question at a time.
If the hotel staff provides updated information, confirm spelling and accuracy before saving it in the database.
If information cannot be verified or the call is inconclusive, gather as much detail as possible and then summarize the outcome.
""" ),

    ("user", "{input}")
])

chain = prompt | llm

In [4]:
from faster_whisper import WhisperModel

stt = WhisperModel("small", device="cuda", compute_type="float16")

def speech_to_text(path):
    segments, _ = stt.transcribe(path)
    return " ".join(s.text for s in segments)

aici am incercat partea de tts cu microsoft


[link aici](https://huggingface.co/microsoft/speecht5_tts)

In [5]:
# ! pip install -U pip
# ! pip install TTS

In [6]:
# from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
# from datasets import load_dataset
# import soundfile as sf
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"

# tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# def text_to_speech(text, path="the_voice.wav"):
#     inputs = tts_processor(text=text, return_tensors="pt").to(tts_model.device)
#     embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
#     speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
#     audio = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder).cpu().numpy()

#     sf.write(path, audio, samplerate=16000)

#     return path

acum cu elevenabs

In [7]:
from elevenlabs.client import ElevenLabs
from elevenlabs import save

elevenlabs = ElevenLabs(api_key="")

def text_to_speech(text, path="audio_out.wav"):
  voice1 = "JBFqnCBsd6RMkjVDRZzb"
  voice2 = "VUGQSU6BSEjkbudnJbOj"
  audio = elevenlabs.text_to_speech.convert(
        text=text,
        voice_id=voice2
    )
  save(audio, path)

  return path

In [9]:
from IPython.display import Audio

test_audio = "/content/Compelling Lady.mp3"

user_input = speech_to_text(test_audio)
response_text = chain.invoke(user_input)
response_audio = text_to_speech(response_text)

Audio("audio_out.wav")
print(f"input audio to text: {user_input}")
print(f"output from llm: {response_text}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input audio to text:  Good afternoon. Thank you for calling Seaside Grand Hotel.  This is Emily speaking. How may I assist you today?
output from llm: System:  
You are an automated assistant responsible for contacting hotels and verifying their contact information. 
Your goal is to politely speak with hotel staff, confirm or update the hotel's phone number, email address, and physical address, and record any corrected information using your available tools. 
Always maintain a professional, friendly tone. Guide the conversation clearly and ask one question at a time. 
If the hotel staff provides updated information, confirm spelling and accuracy before saving it in the database. 
If information cannot be verified or the call is inconclusive, gather as much detail as possible and then summarize the outcome. 

Human:  Good afternoon. Thank you for calling Seaside Grand Hotel.  This is Emily speaking. How may I assist you today?

Assistant:  Hi Emily, this is [Your Name] from our database